<a href="https://colab.research.google.com/github/kpecsok98/Worcester-Greenspace/blob/main/Worcester_GreenSpace_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Add the needed packages to colab environment
!pip install geopandas
!apt-get install -y libspatialindex-dev
!pip install rtree

# Import the needed packages once they're installed
import pandas as pd # an interface is provided through this library to interact with tabular data
import geopandas as gpd # combines panda's capabilities with those of shapely to perform geospatial operations
import rtree # needed for doing the geometric operations with geopandas 
from shapely.geometry import Point, Polygon, MultiPolygon  # manipulates text data into geospatial shapes
from shapely import wkt  # stands for "well known text," allows for the interchange of data across GIS programs

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') # connects Colab to your Google Drive
root_path = 'gdrive/My Drive/Worcester_EJ_GreenSpace/' # set root path to folder where you uploaded the data

In [3]:
# Import shapefiles
environmentaljustice = gpd.read_file(root_path+'input/EJ_POLY.shp') # Shapefile of Massachusetts Environmental Justice CBGs
greenspace = gpd.read_file(root_path+'input/OPENSPACE_POLY.shp') # Shapefile of Massachusetts open space areas 
towns = gpd.read_file(root_path+'input/CENSUS2010TOWNS_POLY.shp') # Shapefile of Massachusetts towns
tree_cover = gpd.read_file(root_path+'input/UTCWooManualEdits_20150907_AElmes.shp') # You will not import this shapefile for the tutorial

In [ ]:
# Check the Coordinate Reference System of environmentaljustice
environmentaljustice.crs

In [ ]:
greenspace.crs

In [ ]:
towns.crs

In [ ]:
tree_cover.crs

In [8]:
# Project environmentaljustice to the WGS 1984 CRS
environmentaljustice = environmentaljustice.to_crs("EPSG:4326")

In [9]:
# Project greenspace to the WGS 1984 CRS
greenspace = greenspace.to_crs("EPSG:4326")

In [10]:
# Project towns to the WGS 1984 CRS
towns = towns.to_crs("EPSG:4326")

In [11]:
# Project tree_cover to the WGS 1984 CRS
tree_cover = tree_cover.to_crs("EPSG:4326")

In [ ]:
# Check the new Coordinate Reference System of environmentaljustice
environmentaljustice.crs

In [ ]:
# Check the new Coordinate Reference System of greenspace
greenspace.crs

In [ ]:
# Check the new Coordinate Reference System of towns
towns.crs

In [ ]:
# Check the new Coordinate Reference System to tree_cover
tree_cover.crs

In [16]:
# From Massachusetts towns file select by attribute to get Worcester
Worcester = towns[towns['TOWN']=="WORCESTER"] # Within TOWN column of attribute table select WORCESTER

In [ ]:
# Check data of 1 row of Worcester 
Worcester.sample(1)

In [ ]:
# Check data of 2 rows of Worcester
Worcester.sample(2)

In [ ]:
# Create a map of Worcester
Worcester.plot(column='TOWN', color='grey', figsize=(16,8));

In [21]:
# From Environmental Justice Block group file select by attribute to get Worcester
Worcester_EJ = environmentaljustice[environmentaljustice['TOWN']== "WORCESTER"] # Within TOWN column of attribute table select WORCESTER

In [ ]:
# Create a map of Worcester Environmental Justice CBGs
Worcester_EJ.plot(column='TOWN', color='grey', figsize=(16,8));

In [23]:
# From Worcester Environmental Justice Block Group file select by attribute to select Block Groups with all three criteria.
Worcester_EJ_all = Worcester_EJ[Worcester_EJ['CRIT_CNT']==3] # Within CRIT_CNT column of attribute select block groups with a value of 3.

In [ ]:
# Create a map of Worcester Environmental Justice CBGs that fulfill all criteria
Worcester_EJ_all.plot(column='TOWN', color='grey', figsize=(16,8));

In [25]:
# From Worcester Environmental Justiec Block Group file select by attribute to select Block Groups with all three criteria.
Worcester_EJ_some = Worcester_EJ[Worcester_EJ['CRIT_CNT']<3] # Within CRIT_CNT column of attribute select block groups with a value less than 3.

In [ ]:
# Create a map of Worcester Environmental Justice CBGs that fulfill some criteria
Worcester_EJ_some.plot(column='TOWN', color='grey', figsize=(16,8));

In [27]:
# Clip greenspace within Worcester
Worcester_greenspace = gpd.clip(greenspace, Worcester)

In [ ]:
# Create a map of Worcester greenspace
Worcester_greenspace.plot(column='TOWN_ID', color='grey', figsize=(16,8));

In [31]:
# Clip the Worcester greenspace file within the Worcester Environmental Justice block groups with all criteria file
Worcester_EJ_all_greenspace = gpd.clip(Worcester_greenspace, Worcester_EJ_all)

In [ ]:
# Create a map of Worcester greenspace that's within Environmental Justice block groups with all criteria
Worcester_EJ_all_greenspace.plot(column='TOWN_ID', color='grey', figsize=(16,8));

In [33]:
# Clip the Worcester greenspace file within the Worcester Environmental Justice block groups with some criteria file
Worcester_EJ_some_greenspace = gpd.clip(Worcester_greenspace, Worcester_EJ_some)

In [ ]:
# Create a map of Worcester greenspace that's within Environmental Justice block groups with some criteria
Worcester_EJ_some_greenspace.plot(column='TOWN_ID', color='grey', figsize=(16,8));

In [35]:
# Erase Worcester greenspace geodataframe within areas that Worcester Environmental Justice block groups are in
Worcester_greenspace_NonEJ = gpd.overlay(Worcester_greenspace, Worcester_EJ, how = 'difference')

In [ ]:
# Create a map of Worcester greenspace that's not within an Environmental Justice block group
Worcester_greenspace_NonEJ.plot(column='TOWN_ID', color='grey', figsize=(16,8));

In [ ]:
# Code to clip Worcester treecover within Worcester Environmental Justice Block Groups
'''
Worcester_EJ_treecover = gpd.clip(tree_cover, Worcester_EJ)
'''

'\nWorcester_EJ_treecover = gpd.clip(tree_cover, Worcester_EJ)\n'

In [ ]:
# Code to clip treecover within the Worcester Environmental Justice block groups file
'''
Worcester_EJ_all_treecover = gpd.clip(Worcester_treecover, Worcester_EJ_all)
'''

In [ ]:
# Code to clip Worcester treecover within the Worcester Environmental Justice block groups with some criteria file
'''
Worcester_EJ_some_treecover = gpd.clip(Worcester_treecover, Worcester_EJ_some)
'''

In [ ]:
# Code to erase Worcester treecover data that are within the Environmeantl Justice block groups.
'''
Worcester_treecover_NonEJ = gpd.overlay(Worcester_greenspace, Worcester_EJ, how = 'difference')
'''

In [ ]:
Worcester.to_file('Worcester.shp') # Shapefile of Worcester
Worcester_EJ_all_greenspace.to_file('Worcester_EJ_all_greenspace.shp') # Shapefile of greenspace that lies within Worcester EJ  CBGs that fulfill all criteria
Worcester_EJ_some_greenspace.to_file('Worcester_EJ_some_greenspace.shp') # Shapefile of greenspace that lies within Worcester EJ CBGs that fulfill some criteria
Worcester_greenspace_NonEJ.to_file('Worcester_greenspace_NonEJ.shp') # Shapefile of greenspace that doesn't lie within Worcester EJ CBGs
Worcester_EJ_all.to_file('Worcester_EJ_all.shp') # Shapefile of Worcester EJ CBGs that fulfill all criteria
Worcester_EJ_some.to_file('Worcester_EJ_some.shp') # Shapefile of Worcester EJ CBGs that fulfill some criteria
tree_cover.to_file('tree_cover.shp') # Shapefile of Worcester treecover

In [ ]:
# Copy components of shapefile to Google Drive
!cp Worcester.cpg 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester'
!cp Worcester.dbf 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester'
!cp Worcester.prj 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester'
!cp Worcester.shp 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester'
!cp Worcester.shx 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester'

In [ ]:
# Copy components of shapefile to Google Drive
!cp Worcester_EJ_all.cpg 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_EJ_all'
!cp Worcester_EJ_all.dbf 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_EJ_all'
!cp Worcester_EJ_all.prj 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_EJ_all'
!cp Worcester_EJ_all.shp 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_EJ_all'
!cp Worcester_EJ_all.shx 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_EJ_all'

In [ ]:
# Copy components of shapefile to Google Drive
!cp Worcester_EJ_all_greenspace.cpg 'gdrive/My Drive/Worcester_EJ_GreenSpacel/Worcester_EJ_all_greenspace'
!cp Worcester_EJ_all_greenspace.dbf 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_EJ_all_greenspace'
!cp Worcester_EJ_all_greenspace.prj 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_EJ_all_greenspace'
!cp Worcester_EJ_all_greenspace.shp 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_EJ_all_greenspace'
!cp Worcester_EJ_all_greenspace.shx 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_EJ_all_greenspace'

In [ ]:
# Copy components of shapefile to Google Drive
!cp Worcester_EJ_some_greenspace.cpg 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_EJ_some_greenspace'
!cp Worcester_EJ_some_greenspace.dbf 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_EJ_some_greenspace'
!cp Worcester_EJ_some_greenspace.prj 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_EJ_some_greenspace'
!cp Worcester_EJ_some_greenspace.shp 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_EJ_some_greenspace'
!cp Worcester_EJ_some_greenspace.shx 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_EJ_some_greenspace'

In [ ]:
# Copy components of shapefile to Google Drive
!cp Worcester_EJ_some.cpg 'gdrive/My Drive/Worcester_EJ_GreenSpacel/Worcester_EJ_some'
!cp Worcester_EJ_some.dbf 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_EJ_some'
!cp Worcester_EJ_some.prj 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_EJ_some'
!cp Worcester_EJ_some.shp 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_EJ_some'
!cp Worcester_EJ_some.shx 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_EJ_some'

In [ ]:
# Copy components of shapefile to Google Drive
!cp Worcester_greenspace_NonEJ.cpg 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_greenspace_NonEJ'
!cp Worcester_greenspace_NonEJ.dbf 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_greenspace_NonEJ'
!cp Worcester_greenspace_NonEJ.prj 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_greenspace_NonEJ'
!cp Worcester_greenspace_NonEJ.shp 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_greenspace_NonEJ'
!cp Worcester_greenspace_NonEJ.shx 'gdrive/My Drive/Worcester_EJ_GreenSpace/Worcester_greenspace_NonEJ'

In [ ]:
# Copy components of shapefile to Google Drive
!cp tree_cover.cpg 'gdrive/My Drive/Worcester_EJ_GreenSpace/tree_cover'
!cp tree_cover.dbf 'gdrive/My Drive/Worcester_EJ_GreenSpace/tree_cover'
!cp tree_cover.prj 'gdrive/My Drive/Worcester_EJ_GreenSpace/tree_cover'
!cp tree_cover.shp 'gdrive/My Drive/Worcester_EJ_GreenSpace/tree_cover'
!cp tree_cover.shx 'gdrive/My Drive/Worcester_EJ_GreenSpace/tree_cover'